# **STATS 503, Group Work Assignment 5: Cross validation**

**Instructions:** During lab section, and afterward as necessary, you will collaborate in two-person teams (assigned by the GSI) to complete the problems that are interspersed below. The GSI will help individual teams encountering difficulty, make announcements addressing common issues, and help ensure progress for all teams. **During lab, feel free to flag down your GSI to ask questions at any point!** Upon completion, one member of the team should submit their team's work through Canvas as html.

**credits:** Roman Kouznetsov, adapted from notes from Gang Qiao.

# Getting started



In this assignment, we will explore logistic regression and $k$-fold cross validation. In $k$-fold cross-validation, we partition a dataset into $k$ equally sized non-overlapping subsets $S$. For each subset $S_i$, a model is trained on $S\backslash S_i$ and evaluated on $S_i$. The cross-validation estimator of the prediction's error is the average of the prediction errors obtained on each fold.

Let's start with simple example that will help us understand how CV works. We'll first import the relevant packages.

In [ ]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import sklearn.metrics
import numpy as np
import pandas as pd

For demonstration purposes, let's create a synthetic binary classification dataset using scikit-learn's `make_classification` function.

In [ ]:
# Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

We'll look at two ways to do cross-validation.  First we'll do it "by hand." Then we'll show how sklearn makes it easy.

# K-fold CV by hand

First, we divide the data into 10 parts (a.k.a. folds).

In [ ]:
n_splits = 10 # 10-fold
rng = np.random.default_rng(0) # make a random number generator with fixed seed
permutation = rng.permutation(len(X)) # create a shuffling of the indices of our data
splits = np.split(permutation,n_splits) # make folds

In [ ]:
# e.g., these are the *indices* of the datapoints "fold 3"
splits[3]

array([290, 277, 360, 524, 462, 789, 908, 493, 818, 251, 623, 123, 781,
       589, 874, 304, 869, 746, 576, 811, 433, 837,  37, 161,  58, 560,
       643, 965, 163,  64, 475, 838, 168, 659, 109,  41, 756, 596, 303,
       763, 101, 703, 120, 531, 366, 553, 548, 575, 731, 470, 578, 660,
       551, 417, 716, 911, 227, 587, 872, 185, 513, 866, 321, 418, 760,
       820, 119, 952,  24, 606, 170, 278, 514,  15, 155, 367, 828,  92,
        55, 897, 118, 791, 701, 145, 860, 645, 620, 778, 497, 794, 246,
       842,  80, 646, 225, 951, 482,  20, 343, 250])

In [ ]:
# and these are the y values for the corresponding samples
y[splits[3]]

array([1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0])

Now, we'll assess predictive performance 10 times.  Each time we'll hold out a different fold and use the rest of the data for training.

In [ ]:
# we'll look at two metrics of predictive performance,
# and store the results in these arrays
missclass = np.zeros(len(splits))
aurocs = np.zeros(len(splits)) # since this is binary classification, we can assess via auroc

# iterate through folds
for i in range(len(splits)):
    # create test/train split for this held-out fold
    s = list(splits) # copy list
    test = s.pop(i) # pop out the ith fold
    training = np.concatenate(s) # combine all remaining data for test

    # fit model
    model = LogisticRegression() # make estimator
    model.fit(X[training], y[training]) # fit estimator using the training data

    # assess metric for predictive performance using test data
    missclass[i] = np.mean(model.predict(X[test])!= y[test]) # get misclassification on test data
    aurocs[i] = sklearn.metrics.roc_auc_score(y[test], model.predict_proba(X[test])[:, 1]) # get auroc on test data

In [ ]:
missclass # misclassification rate for each of the 10 test/train splits

array([0.15, 0.15, 0.16, 0.1 , 0.21, 0.13, 0.11, 0.09, 0.15, 0.12])

In [ ]:
aurocs # auroc for each of the 10 test/train splits

array([0.89783654, 0.90381494, 0.91304348, 0.93427036, 0.91666667,
       0.90868506, 0.9696    , 0.96025692, 0.93830128, 0.95392628])

In [ ]:
# report conclusions: mean(scores) pm sqrt(tau^2/K)
print(f"Mean misclassification rate: {missclass.mean():.4f}, plus minus {np.sqrt(missclass.var()/n_splits):.4f}")
print(f"Auroc: {aurocs.mean():.4f}, plus minus {np.sqrt(aurocs.var()/n_splits):.4f}")

Mean misclassification rate: 0.1370, plus minus 0.0105
Auroc: 0.9296, plus minus 0.0076


# K-fold CV using `sklearn.model_selection`

In [ ]:
model = LogisticRegression()

Setup the $k$-fold cross-validation configuration. For example, using 10 folds...
* `n_splits=10` means 10-fold
* `shuffle=True` meaning random folds
* `random_state=42` means we'll get the same random folds each time

In [ ]:
cv = KFold(n_splits=10, random_state=42, shuffle=True)

In [ ]:
# look at folds generated by this cross-validation splitter
for i, (train, test) in enumerate(cv.split(X)):
    # this automatically gives you the train indices and test indices
    # without having to construct them yourselves by combining folds
    print(f'\nsplit {i}')
    print('   test:   [', ' '.join(test[:10].astype('U')),'...]') # test indices
    print('   train:  [', ' '.join(train[:10].astype('U')),'...]') # train indices


split 0
   test:   [ 10 23 30 39 54 59 63 66 67 70 ...]
   train:  [ 0 1 2 3 4 5 6 7 8 9 ...]

split 1
   test:   [ 25 44 55 60 72 78 86 110 120 137 ...]
   train:  [ 0 1 2 3 4 5 6 7 8 9 ...]

split 2
   test:   [ 2 3 5 7 9 29 31 33 49 65 ...]
   train:  [ 0 1 4 6 8 10 11 12 13 14 ...]

split 3
   test:   [ 0 6 28 41 56 69 73 79 90 108 ...]
   train:  [ 1 2 3 4 5 7 8 9 10 11 ...]

split 4
   test:   [ 11 12 18 24 42 43 51 61 74 83 ...]
   train:  [ 0 1 2 3 4 5 6 7 8 9 ...]

split 5
   test:   [ 15 19 22 38 46 50 57 68 75 93 ...]
   train:  [ 0 1 2 3 4 5 6 7 8 9 ...]

split 6
   test:   [ 8 16 17 26 36 37 45 48 53 103 ...]
   train:  [ 0 1 2 3 4 5 6 7 9 10 ...]

split 7
   test:   [ 112 122 123 129 143 146 147 183 186 197 ...]
   train:  [ 0 1 2 3 4 5 6 7 8 9 ...]

split 8
   test:   [ 4 14 27 32 35 40 47 52 62 64 ...]
   train:  [ 0 1 2 3 5 6 7 8 9 10 ...]

split 9
   test:   [ 1 13 20 21 34 58 71 80 87 91 ...]
   train:  [ 0 2 3 4 5 6 7 8 9 10 ...]


In fact, you don't even have to run the for loop yourself!

With a single call to cross_val_score, we can evaluate the model using cross-validation. Here, we'll use accuracy as the performance metric, but you can choose other metrics like precision, recall, etc.

In [ ]:
# Calculate accuracies across folds
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores

array([0.87, 0.86, 0.83, 0.89, 0.88, 0.86, 0.85, 0.87, 0.9 , 0.86])

In [ ]:
print(f"Mean accuracy: {scores.mean():.4f}, plus minus {np.sqrt(scores.var()/len(scores)):.4f}")
print(f"Mean misclassification rate: {1-scores.mean():.4f}, plus minus {np.sqrt(scores.var()/len(scores)):.4f}")

Mean accuracy: 0.8670, plus minus 0.0060
Mean misclassification rate: 0.1330, plus minus 0.0060


Note that conclusions are not exactly the same, because CV used different splits.

If you want to use multiple metrics, use skelarn.cross_validate instead.

In [ ]:
# info for each split
results = pd.DataFrame(
    sklearn.model_selection.cross_validate(model, X, y, cv=cv,
                                           scoring=('accuracy','roc_auc'),
                                           return_train_score=True)
    )
results

,fit_time,score_time,test_accuracy,train_accuracy,test_roc_auc,train_roc_auc
0,0.011338,0.005495,0.87,0.874444,0.910879,0.937207
1,0.004352,0.005089,0.86,0.880000,0.926250,0.936828
2,0.004015,0.004012,0.83,0.881111,0.900641,0.939317
3,0.007936,0.004390,0.89,0.872222,0.942222,0.934140
4,0.004344,0.004903,0.88,0.877778,0.951600,0.933526
5,0.004102,0.004144,0.86,0.876667,0.939382,0.934303
6,0.004033,0.004348,0.85,0.883333,0.910364,0.937373
7,0.006532,0.005003,0.87,0.880000,0.890405,0.939992
8,0.004055,0.004431,0.90,0.870000,0.961851,0.932240
9,0.009408,0.004527,0.86,0.876667,0.958788,0.932327


In [ ]:
print(f"Auroc: {results.test_roc_auc.mean():.4f}, plus minus {np.sqrt(results.test_roc_auc.var()/len(results)):.4f}")
print(f"Accuracy: {results.test_accuracy.mean():.4f}, plus minus {np.sqrt(results.test_accuracy.var()/len(results)):.4f}")

Auroc: 0.9292, plus minus 0.0080
Accuracy: 0.8670, plus minus 0.0063


Note the results also include train performance (because we set `return_train_score=True`).  Can sometimes be interesting to see discrepancy between train and test performance.  Usually train performance is a bit better.  If train performance is *a lot* better your estimator may have "too much flexibility" (though sometimes you may also be experiencing so-called "benign overfitting" in which case your estimator is actually just fine...).  

In [ ]:
print(f"Train auroc: {results.train_roc_auc.mean():.4f}, plus minus {np.sqrt(results.train_roc_auc.var()/len(results)):.4f}")
print(f"Train accuracy: {results.train_accuracy.mean():.4f}, plus minus {np.sqrt(results.train_accuracy.var()/len(results)):.4f}")

Train auroc: 0.9357, plus minus 0.0009
Train accuracy: 0.8772, plus minus 0.0013


# Compare with another estimator (using same splits!)

In [ ]:
model2 =  KNeighborsClassifier(5)
scores2 = cross_val_score(model2, X, y, scoring='accuracy', cv=cv,n_jobs=-1) # note, using same folds, cv

print(f"Mean Accuracy: {scores2.mean():.4f}, plus minus {np.sqrt(scores2.var()/len(scores2)):.4f}")

Mean Accuracy: 0.8120, plus minus 0.0110


It seems that 5-NN is worse, by a margin that is well in excess of the spread.

# Bias-variance tradeoff

It is very helpful to think about bias-variance tradeoff in cross-validation. In CV, the number of folds to use (the value of $k$) is an important decision. Imagine repeating the learning procedure on multiple datasets. The lower the value for $k$, the higher the bias in the error estimates and the less variance **across datasets**. Conversely, when $k$ is set equal to the training+val sample size, the error estimate is then very low in bias but has the possibility of high variance **across datasets**.

Why? Some intuitions (but not mathematically rigorous proof) here:

While there is no overlap between the test sets on which the models are evaluated, there is overlap between the training sets for all $k>2$. The overlap is largest for leave-one-out cross-validation. This means that the learned models are correlated, i.e. dependent, and the variance of the sum of correlated variables increases with the amount of covariance:
$
Var(\sum_i X_i) = \sum_i \sum_j Cov(X_i, X_j).
$
Therefore, leave-one-out cross-validation has large variance in comparison to CV with smaller $k$. To summarize, larger $k$ means less bias towards overestimating the true expected error (as training folds will be closer to the total dataset) but higher variance and higher running time (as you are getting closer to the limit case: Leave-One-Out CV).

For more fun facts and simulation about bias-variance tradeoff and cross validation, please see [this post](https://stats.stackexchange.com/questions/61783/bias-and-variance-in-leave-one-out-vs-k-fold-cross-validation/357749#357749).

# Group Work 5 (30 points)

But WHY do we even bother with cross-validation? What is the point?

In this group work assignment, you'll perform K-fold cross validation on several classification models on the NHANES dataset. Additionally, you will be asked to write answers to explain WHY we do the things we do. Please feel free to ask instructors for guidance if cross-validation is still new to you.



---


### Part 1: Dataset Setup (1 point)

Our favorite (and only) dataset we have used in the group work assignments is back! Please take the appropriate time to load in the 3 NHANES datasets.

In [ ]:
# df =
bmx_df = pd.read_sas('BMX_L.xpt', format = 'xport')
demo_df = pd.read_sas('DEMO_L.xpt', format = 'xport')
hdl_df = pd.read_sas('HDL_L.xpt', format = 'xport')

df = pd.merge(hdl_df, bmx_df, on = 'SEQN', how = "inner")
df = pd.merge(df, demo_df, on = 'SEQN', how = "inner")
df.head()

,SEQN,WTPH2YR,LBDHDD,LBDHDDSI,BMDSTATS,BMXWT,BMIWT,BMXRECUM,BMIRECUM,BMXHEAD,...,DMDHRGND,DMDHRAGZ,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVSTRA,SDMVPSU,INDFMPIR
0,130378.0,56042.129410,45.0,1.16,1.0,86.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,50055.450807,54374.463898,173.0,2.0,5.00
1,130379.0,37435.705647,60.0,1.55,1.0,101.8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,29087.450605,34084.721548,173.0,2.0,5.00
2,130380.0,85328.844519,49.0,1.27,1.0,69.4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,80062.674301,81196.277992,174.0,1.0,1.41
3,130386.0,44526.214135,46.0,1.19,1.0,90.6,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,30995.282610,39988.452940,179.0,1.0,1.33
4,130387.0,22746.296353,42.0,1.09,1.0,103.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,19896.970559,20776.254850,181.0,1.0,1.32




---


### Part 2: Variable Setup and Selection (3 points)

For this problem, we will again try to predict individuals that have high-density lipoprotein (HDL) cholesterol of greater than 60. An HDL of 60 **mg/dL** or higher is often viewed as protective against heart disease—this is typically the level you’d like to aim for, if possible. For this task please do the following:

1. Create the binary indicator variable. Also, use the following features for predictive purposes: Gender, Age, Weight, Height, BMI, WaistSize, Household Size, and Ethnicity. You may need to refer to the docs to figure out their variable names.



*   [HDL_L](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/HDL_L.htm)
*   [DEMO_L](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DEMO_L.htm)
*   [BMX_L](https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BMX_L.htm)



2. **Please** change the variable names to be English-legible but still python variable style.

3. Drop all missing values.

In [ ]:
selected_columns = ["LBDHDD", "RIAGENDR", "RIDAGEYR", "BMXWT", "BMXHT", "BMXBMI", "BMXWAIST", "DMDHHSIZ", "RIDRETH1"]
filtered_data = df[selected_columns].copy()
my_df = filtered_data.rename(columns={
    "LBDHDD": "HDL",
    "RIAGENDER": "Gender",
    "RIDAGEYR": "Age",
    "BMXWT": "Weight",
    "BMXHT": "Height",
    "BMXBMI": "BMI",
    "BMXWAIST": "WaistSize",
    "DMDHHSIZ": "HouseholdSize",
    "RIDRETH1": "Ethnicity"
})
my_df["HDL"] = my_df["HDL"] > 60.0
my_df = my_df.rename(columns={"HDL": "HDL>60"})
my_df = my_df.dropna()
my_df.head()


,HDL>60,RIAGENDR,Age,Weight,Height,BMI,WaistSize,HouseholdSize,Ethnicity
0,False,1.0,43.0,86.9,179.5,27.0,98.3,4.0,5.0
1,False,1.0,66.0,101.8,174.2,33.5,114.7,2.0,3.0
2,False,2.0,44.0,69.4,152.9,29.7,93.5,7.0,2.0
3,False,1.0,34.0,90.6,173.3,30.2,106.1,3.0,1.0
4,False,2.0,68.0,103.5,155.9,42.6,122.0,1.0,3.0


In [ ]:
assert my_df.shape[0] / df.shape[0] > 0.95



---


### Part 3: Training + Testing Split (1 point)

Please split your data into a train and test set, with 70% of observations in the train set and 30% in the test set.

*   Please use the `train_test_split` method.
*   Please use the `random_state=42`.
*   Please stratify the sampling to include roughly the same distribution of response values in each set. [Why should we stratify?](https://scikit-learn.org/stable/modules/cross_validation.html#stratified-k-fold)


In [ ]:
X = my_df.drop(columns=["HDL>60"])
y = my_df["HDL>60"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)



---


### Part 4: Implementing K-fold CV (2 points)

Write a function called KFoldCV that takes in as input 4 arguments:

1.   `X` The predictors array.
2.   `y` The response variable array.
3.   `model`: An sklearn model object on which we can call fit and predict.
4.   `K`: An integer representing the number of folds we want to include.

and returns the list of classification accuracies for each fold.



In [ ]:
def KFoldCV(X, y, model, K=10):

  cv = KFold(n_splits=K, random_state=42, shuffle=True)
  scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  return scores



---


### Part 5 (Code): Execution (2 points)

Run your KFoldCV function using Logistic Regression with $K=10$.

Please ensure the that the output of the function is visible in the notebook.

<u>Note</u>: In the event you get a warning that the model has not reached convergence, add the argument `max_iter=1000` to the LogisticRegression instance.

In [ ]:
KFoldCV(X_train, y_train, LogisticRegression(max_iter=1000), 10)

array([0.7936803 , 0.76951673, 0.74349442, 0.79702048, 0.74674115,
       0.7877095 , 0.80446927, 0.78212291, 0.79702048, 0.77839851])

### Part 5 (Writing): Generalization (2 points)

One motivation for cross-validation is to assess the generalizability of your model on unseen data.

Explain in at most two sentences whether you believe your model generalizes well on unseen data. Reference the output of your function to make your case.

**ANSWER:**

I believe out model generalizes well on unseen data as we have similar results from each fold in the above case and for each fold, the accuracy seems relatively high and reasonable.



---


### Part 6 (Code): Working with Regularization (3 points)

So turns out, if you look at the LogisticRegression module in sklearn, it uses an L2 penalty by default! You can see for yourself on the [documentation page for sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

One problem, we just accepted the default regularization term: C=1.0. How do we know this was the right way to go? Let's see if it even is.

Write a function that performs K-Fold validation but with each fold having a different value for $C$. The values considered should range be $10^{-5}, 10^{-4}, \dots 10^{4}$.

In [ ]:
from sklearn.metrics import accuracy_score
# function returns dictionary
def KFoldCV_L2(X, y, K=10) -> dict:
    """
    Computes mean accuracy using KFold CV for several regularization values.

    Returns:
    dict: key-value pairs of regularization value and mean accuracy.
    """
    results = {}
    regularization_values = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000, 10000]
    for i in range(len(regularization_values)):
        cv = KFold(n_splits=K, random_state=42, shuffle=True)
        model = LogisticRegression(C=regularization_values[i], max_iter=1000)
        scores = cross_val_score(model, X, y, scoring='accuracy', cv=K, n_jobs=-1)
        mean_accuracy = np.mean(scores)
        results[regularization_values[i]] = mean_accuracy

    return results

In [ ]:
KFoldCV_L2(X_train, y_train)

{1e-05: 0.7712636636137706,
 0.0001: 0.7740534983697119,
 0.001: 0.777215772604238,
 0.01: 0.7770302451316344,
 0.1: 0.7796393982817942,
 1: 0.7792669588032093,
 10: 0.7794528324091573,
 100: 0.7796390521484496,
 1000: 0.7796390521484496,
 10000: 0.7796390521484496}

### Part 6 (Writing): Evaluation (2 points)

Based on your cross-validation, do you believe that regularization plays an effect on the predictive accuracy on unseen accuracy? What level of regularization should we choose?

**ANSWER:**

$$\boxed{\textrm{
### According to my output above, the regularization does have an effect on the predictive accuracy. Based on the output, we choose $c=0.1$ as the regularization level at which we get the highest mean accuracy.
}}$$



---


### Part 7 (Code): Train+Val+Test (3 points)

Please now retrain your final model (with the best regularization value you found in the last part) on all training data. Then, evaluate your model's performance on the test set.

In [ ]:
best_C = 0.1
new_model =LogisticRegression(C=best_C, max_iter=1000)
new_model.fit(X_train, y_train)

y_pred = new_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7907986111111112


### Part 7 (Writing 1): Retraining on Training + Validation Data (1 point)

Explain in 1-2 sentences **MAX** why we retrain the model on the merged train+val dataset.

**ANSWER:**

$$\boxed{\textrm{
We retrain the model on the merged train+val dataset because it can utilize all available datasets to improve the model performance, while keeping the test completely independent for the final evaluation.
}}$$

### Part 7 (Writing 2): Evalution (1 point)

Is logistic regression performing well on this data? Explain in 1-2 sentences **MAX**. **You may need some supporting code for your argument**.

<u>Hint</u>: Think about what a trivial baseline would be.

In [ ]:
baseline = max(np.mean(y_train), 1 - np.mean(y_train))
print(f"Baseline accuracy (predicting majority class): {baseline:.4f}")
print(f"Logistic regression accuracy: {accuracy:.4f}")

Baseline accuracy (predicting majority class): 0.7709
Logistic regression accuracy: 0.7908


**ANSWER:**

$$\boxed{\textrm{
Considering the benchmark accuracy rate about 77%, the logistic regression model reaches the accuracy 79%
}}$$



---


### Part 8: K-Folds on K-Nearest Neighbors (2 points)

Using your KFoldCV function you created in Part 4, please run CV on 2-NN classification.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_2 = KNeighborsClassifier(n_neighbors=2)
knn_scores = KFoldCV(X_train, y_train, knn_2, 10)
print("2-NN cross-validation scores:", knn_scores)
print(f"Mean accuracy: {np.mean(knn_scores):.4f}")

2-NN cross-validation scores: [0.75836431 0.77509294 0.73234201 0.77653631 0.72625698 0.77281192
 0.77467412 0.75791434 0.76536313 0.74115456]
Mean accuracy: 0.7581




---


### Part 9: Finding the Right K (4 points)

Similar to what you did for regularized logistic regression, please run a 10-fold CV that assesses model performance on 10 different number of neighbors: [1, 3, 5, 10, 15, 20, 25, 35, 50, 100]. You should attempt to predict each fold with each num_neighbors value.

Please write a function called `KFoldCV_NN` that takes as input `X`, `y`, and `K` and returns a dictionary. The dictionary should have the number of neighbors considered as keys and the mean validation accuracy as values.



In [ ]:
def KFoldCV_NN(X, y, K=10):
    """
    Computes mean accuracy using KFold CV for several K values.

    Returns:
    dict: key-value pairs of regularization value and mean accuracy.
    """
    results = {}
    neighbors = [1, 3, 5, 10, 15, 20, 25, 35, 50, 100]

    for n in neighbors:
        model = KNeighborsClassifier(n_neighbors=n)
        scores = cross_val_score(model, X, y, cv=K, scoring='accuracy')
        results[n] = np.mean(scores)

    return results

In [ ]:
KFoldCV_NN(X_train, y_train)

{1: 0.7016583248530663,
 3: 0.7381373180203943,
 5: 0.7509767882979238,
 10: 0.767915169639952,
 15: 0.7695911472935835,
 20: 0.7753580749447918,
 25: 0.7753577288114473,
 35: 0.7772188878043378,
 50: 0.7777778931555592,
 100: 0.7777789315555925}



---


### Part 10: Retraining on Training + Validation Data (2 points)

Please retrain your data on both the training and validation set. Make sure you run the KNeighborsClassifier with the best number of neighbors as you determined from the previous part.

In [ ]:
nn_results = KFoldCV_NN(X_train, y_train)
best_k = max(nn_results.items(), key=lambda x: x[1])[0]

final_knn = KNeighborsClassifier(n_neighbors=best_k)
final_knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=100)



---


### Part 11: Test Set Evaluation (1 point)

Evaluate your final model's performance.

In [ ]:
knn_test_pred = final_knn.predict(X_test)
knn_test_accuracy = accuracy_score(y_test, knn_test_pred)
print(f"KNN Test set accuracy: {knn_test_accuracy:.4f}")

# compare logistic regression and KNN
print("\nFinal Model Comparison:")
print(f"Logistic Regression accuracy: {accuracy:.4f}")
print(f"KNN accuracy: {knn_test_accuracy:.4f}")

KNN Test set accuracy: 0.7804

Final Model Comparison:
Logistic Regression accuracy: 0.7908
KNN accuracy: 0.7804
